## 撰寫網路爬蟲爬取Yahoo 電影評論

In [2]:
import requests
from bs4 import BeautifulSoup
res = requests.get('https://movies.yahoo.com.tw/movieinfo_review.html/id=10240')
soup = BeautifulSoup(res.text, 'lxml')

In [17]:
ary = []
for com in soup.select('.usercom_list li'):
    comment = com.select('span')[2].text
    stars = com.select_one('input[name="score"]').get('value')
    ary.append({'comment':comment, 'stars':stars})

In [18]:
import pandas
df = pandas.DataFrame(ary)

In [19]:
df

,comment,stars
0,頭一次覺得得獎又超好看的！\r\n是自己心目中今年第一名的國片\r\n很值得進戲院,5
1,最多最多三顆星。不用花錢進來看,3
2,本來只想著支持金馬獎的心情去看，沒想到超級好看!!!!!一開始就有爆點~~~從頭到尾真的沒有...,5
3,不懂評論裡的五顆星是怎麼一回事，邊看邊覺得尷尬癌上身，畫面很美但剪接零碎的讓人出戲，人物刻畫...,1
4,得獎實至名歸 很久沒進戲院了 看完後覺得值回票價 難得的好片,5
5,很好看！最佳劇情長片實至名歸！值得進戲院支持,5
6,家庭真的是很難，或許也很簡單。,5
7,很棒的電影，值得一看再看的好戲！,5
8,社會寫實面，很值得看的國片!!,5
9,很寫實的劇情，確實會是發生在家中的每一個角落\r\n片長158分鐘，確毫無冷場,5


## 讀取Yahoo 電影評論 

In [22]:
import pandas
df = pandas.read_excel('https://raw.githubusercontent.com/ywchiu/pytextmining/master/data/yahoo_movie.xlsx', index_col = 0)

In [23]:
df.head()

,content,stars,title,status
0,"不知道耶! 看完整個無感\r\n有種覺得就是女兒""給蕭""害死了老爸\r\n然後...既然可以...",3,古墓奇兵,soso
1,很好看的動作片，不會浪費錢跟時間。很久沒有這樣的探險片。可說是女版的印第安那瓊。女主角跟爸爸...,5,古墓奇兵,good
2,說這個不好看的話，那我還真不知道，還有什麼片是您可以去看得了。電影好看，但話說羅拉有裝可以撿...,5,古墓奇兵,good
3,父女重逢真的很讓人感動，五顆星。,5,古墓奇兵,good
4,劇情雖然老套，但仍拍出新意，古墓能殺人的方式不就是機關和毒，要求亂七八糟的觀眾，你看喪尸片看多了。,4,古墓奇兵,good


In [25]:
df['title'].unique()

array(['古墓奇兵', '最後的絕地武士', '攻殼機動隊', '奧創紀元', '最終騎士'], dtype=object)

In [28]:
import jieba
corpus = []
tags = []
for idx, rec in df[df['status'].isin(['good', 'bad'])].iterrows():
    corpus.append(' '.join(jieba.cut(rec['content'])))
    tags.append(rec['status'])

Building prefix dict from the default dictionary ...
Dumping model to file cache /var/folders/46/b7dzk4mn6g54qzptv608w7d00000gn/T/jieba.cache
Loading model cost 0.806 seconds.
Prefix dict has been built succesfully.


In [29]:
corpus[0]

'很 好看 的 動作片 ， 不會 浪費 錢 跟 時間 。 很 久 沒有 這樣 的 探險 片 。 可 說 是 女版 的 印第安那 瓊 。 女 主角 跟 爸爸 還 有 反派 都 演 得到 位 。 陸任 的 男 配角 常 在 港片 看到 ， 很帥 。 一時 忘了 名字 。 希望 有 續集 。'

In [30]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(corpus)

In [31]:
X.shape

(980, 4839)

In [32]:
len(tags)

980

In [33]:
from sklearn.model_selection import train_test_split
train_X, test_X, train_y, test_y = train_test_split(X, tags, test_size = 0.3, random_state = 42)

In [34]:
train_X.shape

(686, 4839)

In [35]:
test_X.shape

(294, 4839)

In [36]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB(alpha = 0.01)
clf.fit(train_X, train_y)

MultinomialNB(alpha=0.01, class_prior=None, fit_prior=True)

In [37]:
predicted = clf.predict(test_X)

In [41]:
(predicted == test_y).sum() / len(predicted)

0.7687074829931972

In [42]:
from sklearn.metrics import accuracy_score, confusion_matrix
accuracy_score(test_y, predicted)

0.7687074829931972

In [44]:
print(clf.classes_)

['bad' 'good']


In [43]:
confusion_matrix(test_y, predicted)

array([[ 82,  35],
       [ 33, 144]])

In [46]:
import operator
coef_features_c1_c2 = []

for index, features in enumerate(zip(vectorizer.get_feature_names(), \
                        clf.feature_count_[0], clf.feature_count_[1])):
    feat,c1,c2 = features
    coef_features_c1_c2.append(tuple([c1/(c2 + 1), feat, c1, c2]))

for i in sorted(coef_features_c1_c2, key = operator.itemgetter(0), reverse=True)[0:20]:
    print(i)


(6.672863530338025, '睡著', 10.800754416632653, 0.6186086179534868)
(4.823810509851034, '浪費', 7.204200091270568, 0.49346664355044145)
(4.058824255401444, '爛片', 7.2666613024304265, 0.7903365223956232)
(3.6956875803437925, '看到', 13.897332511559544, 2.760418652668346)
(3.4692918154989485, '睡覺', 4.362408041695186, 0.2574347370279635)
(3.2563387106676434, '失望', 7.196320060507634, 1.2099421159515005)
(3.184964059891438, '演什麼', 3.184964059891438, 0.0)
(2.6264409387388845, '只有', 4.606838953575162, 0.7540234336231402)
(2.2930874637087633, '空洞', 3.1738043456202947, 0.3840747009654352)
(2.17199793254736, '五集', 2.3499882879130665, 0.08194775542762894)
(2.1053163150222374, '四集', 2.1053163150222374, 0.0)
(2.021695889662221, '亂七八糟', 2.529498271257764, 0.25117644260551225)
(1.9198916259333108, '爛透', 1.9198916259333108, 0.0)
(1.9147255861808632, '超爛', 1.9147255861808632, 0.0)
(1.8322298165070485, '難看', 11.96566093557524, 5.530655067270165)
(1.764198580729871, '小時', 2.941518804755412, 0.6673399677821239)


In [59]:
s = '很好看！最佳劇情長片實至名歸！值得進戲院支持'
seg_corpus = [' '.join(jieba.cut(s))]

In [60]:
predict_X = vectorizer.transform(seg_corpus)

In [61]:
clf.predict(predict_X)

array(['good'], dtype='<U4')